#Creating the Dimension Table

In this file, I have computed from and engineered the fact table to include features that are relevant for my opening week revenue prediction.

##Importing necessary packages

In [4]:
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime, timedelta
import os
import sys
import subprocess

##Loading the Dataframes

In [8]:
# 1. AUTO-INSTALL GDOWN (If missing)
try:
    import gdown
except ImportError:
    print("gdown not found. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown

# 2. FILE MAPPING
# Map to the "Google Drive Link
files = {
    "fact_table.csv": "https://drive.google.com/file/d/1VadGjcTHxsU5Wj5LMM8_BaXbfVHLcw5_/view?usp=share_link",
    "title.crew.tsv":    "https://drive.google.com/file/d/1t46FEh43JOUX5o_j53gJ_yoLblsNFLxt/view?usp=share_link"
}

# 3. DOWNLOADER LOOP
for filename, drive_link in files.items():
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")

        # Extract ID from the link safely
        try:
            file_id = drive_link.split('/d/')[1].split('/')[0]
            url = f'https://drive.google.com/uc?id={file_id}'

            # Download (quiet=False shows the progress bar)
            gdown.download(url, filename, quiet=False)
        except IndexError:
            print(f"Error: Could not parse ID for {filename}. Check the link.")
    else:
        print(f"Found {filename} locally. Skipping download.")

# 4. LOAD DATA
print("\nLoading Dataframes...")

# Load Mojo (CSV)
if os.path.exists("fact_table.csv"):
    df = pd.read_csv("fact_table.csv")
    print("Fact table loaded.")

# Load IMDB (TSV)
if os.path.exists("title.crew.tsv"): # Corrected from "title.basics.tsv"
    df_crew = pd.read_csv("title.crew.tsv", sep='\t', low_memory=False)
    print("IMDb crew data loaded.")

Found fact_table.csv locally. Skipping download.
Found title.crew.tsv locally. Skipping download.

Loading Dataframes...


/tmp/ipython-input-337254456.py:38: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fact_table.csv")


Fact table loaded.
IMDb crew data loaded.


##Looking at the datasets

In [6]:
df.head()

,tconst,Release,Daily,date,%¬± YD,%¬± LW,Theaters,Release_Year,Avg,Days,To Date,Distributor,genres,runtimeMinutes,Special_Day
0,tt0822832,Marley & Me,"$9,956,212",2009-01-01,44.60%,-30.80%,"3,480",2009,"$2,860",8,"$82,400,283",Twentieth Century Fox,"Drama,Family",115,New Year's Day
1,tt0960731,Bedtime Stories,"$8,336,917",2009-01-01,46.40%,-21.20%,"3,681",2009,"$2,264",8,"$65,037,829",Walt Disney Studios Motion Pictures,"Adventure,Comedy,Family",99,New Year's Day
2,tt0421715,The Curious Case of Benjamin Button,"$7,939,690",2009-01-01,85.40%,-33.10%,"2,988",2009,"$2,657",8,"$60,605,838",Paramount Pictures,"Drama,Fantasy,Romance",166,New Year's Day
3,tt0985699,Valkyrie,"$5,747,446",2009-01-01,64.80%,-32.30%,"2,711",2009,"$2,120",8,"$46,649,304",United Artists,"Drama,History,Thriller",121,New Year's Day
4,tt1068680,Yes Man,"$5,567,221",2009-01-01,102.30%,-6.10%,"3,434",2009,"$1,621",14,"$65,596,911",Warner Bros.,"Comedy,Romance",104,New Year's Day


Above is my previously created dataset containing daily movie data from 2009 to 2025.

In [9]:
df_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12093921 entries, 0 to 12093920
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   tconst     object
 1   directors  object
 2   writers    object
dtypes: object(3)
memory usage: 276.8+ MB


**title.crew.tsv.gz** is a non-commercial IMDb dataset containing the following:

tconst (string) - alphanumeric unique identifier of the title

directors (array of nconsts) - director(s) of the given title

writers (array of nconsts) – writer(s) of the given title

##Merging the Datasets

I only want the director information from the IMDb dataset so I merge the two files accordingly:

In [ ]:
df = pd.merge(df, df_crew[['tconst', 'directors']], on='tconst', how='left')
display(df.head())

,tconst,Release,Daily,date,%¬± YD,%¬± LW,Theaters,Release_Year,Avg,Days,To Date,Distributor,genres,runtimeMinutes,Special_Day,directors
0,tt0822832,Marley & Me,"$9,956,212",2009-01-01,44.60%,-30.80%,"3,480",2009,"$2,860",8,"$82,400,283",Twentieth Century Fox,"Drama,Family",115,New Year's Day,nm0291205
1,tt0960731,Bedtime Stories,"$8,336,917",2009-01-01,46.40%,-21.20%,"3,681",2009,"$2,264",8,"$65,037,829",Walt Disney Studios Motion Pictures,"Adventure,Comedy,Family",99,New Year's Day,nm0788202
2,tt0421715,The Curious Case of Benjamin Button,"$7,939,690",2009-01-01,85.40%,-33.10%,"2,988",2009,"$2,657",8,"$60,605,838",Paramount Pictures,"Drama,Fantasy,Romance",166,New Year's Day,nm0000399
3,tt0985699,Valkyrie,"$5,747,446",2009-01-01,64.80%,-32.30%,"2,711",2009,"$2,120",8,"$46,649,304",United Artists,"Drama,History,Thriller",121,New Year's Day,nm0001741
4,tt1068680,Yes Man,"$5,567,221",2009-01-01,102.30%,-6.10%,"3,434",2009,"$1,621",14,"$65,596,911",Warner Bros.,"Comedy,Romance",104,New Year's Day,nm0715636


##Engineering Special Days

A lot of movies do not release exactly on a holiday rather a week earlier to capture the holiday crowd (with more free time away from school and work) leading up to the actual day and maximize revenue. To capture that effect, I am assigning the same value to one whole week before any special days.

In [ ]:
df['Special_Day'].unique()

array(["New Year's Day", nan, 'Martin Luther King Jr. Day',
       "Washington's Birthday", 'Memorial Day',
       'Independence Day (observed)', 'Independence Day', 'Labor Day',
       'Columbus Day', 'Veterans Day', 'Thanksgiving Day',
       'Christmas Day', 'Christmas Day (observed)',
       "New Year's Day (observed)", 'Veterans Day (observed)',
       'Juneteenth National Independence Day (observed)',
       'Juneteenth National Independence Day'], dtype=object)

In [ ]:
mask = df['Special_Day'].str.contains('(observed)', na=False) #Excluded to avoid duplication
df.loc[mask, 'Special_Day'] = np.nan

/tmp/ipython-input-523161326.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['Special_Day'].str.contains('(observed)', na=False)


In [ ]:
(df['Special_Day'].unique())

array(["New Year's Day", nan, 'Martin Luther King Jr. Day',
       "Washington's Birthday", 'Memorial Day', 'Independence Day',
       'Labor Day', 'Columbus Day', 'Veterans Day', 'Thanksgiving Day',
       'Christmas Day', 'Juneteenth National Independence Day'],
      dtype=object)

Now, I calculate to cover the prior week to any Special Day under the same consideration.

In [ ]:
# Helper function to find the nth occurrence of a weekday in a month
def nth_weekday(year, month, weekday, n):
    """
    weekday: 0=Monday, 1=Tuesday, ..., 6=Sunday
    n: 1=first, 2=second, etc.
    """
    first_day = datetime(year, month, 1)
    days_ahead = (weekday - first_day.weekday()) % 7
    first_occurrence = first_day + timedelta(days=days_ahead)
    nth_occurrence = first_occurrence + timedelta(weeks=n - 1)
    # Ensure it doesn't spill into next month (though for our holidays, it won't)
    if nth_occurrence.month != month:
        return None
    return nth_occurrence

# Helper function for last weekday of the month
def last_weekday_of_month(year, month, weekday):
    """
    Find the last occurrence of weekday in the given month.
    """
    # Get last day of the month
    if month == 12:
        last_day = datetime(year + 1, 1, 1) - timedelta(days=1)
    else:
        last_day = datetime(year, month + 1, 1) - timedelta(days=1)
    days_back = (last_day.weekday() - weekday) % 7
    return last_day - timedelta(days=days_back)

# Function to get all US federal holidays for a given year (actual dates, not observed)
def get_holidays(year):
    holidays = {}
    # Fixed dates
    holidays["New Year's Day"] = datetime(year, 1, 1)
    holidays["Independence Day"] = datetime(year, 7, 4)
    holidays["Veterans Day"] = datetime(year, 11, 11)
    holidays["Christmas Day"] = datetime(year, 12, 25)

    # Variable dates
    holidays["Martin Luther King Jr. Day"] = nth_weekday(year, 1, 0, 3)  # 3rd Monday in January
    holidays["Washington's Birthday"] = nth_weekday(year, 2, 0, 3)  # 3rd Monday in February
    holidays["Memorial Day"] = last_weekday_of_month(year, 5, 0)  # Last Monday in May
    if year >= 2021:
        holidays["Juneteenth National Independence Day"] = datetime(year, 6, 19)
    holidays["Labor Day"] = nth_weekday(year, 9, 0, 1)  # 1st Monday in September
    holidays["Columbus Day"] = nth_weekday(year, 10, 0, 2)  # 2nd Monday in October
    holidays["Thanksgiving Day"] = nth_weekday(year, 11, 3, 4)  # 4th Thursday in November

    return holidays

# Function to update Special_Day for a row
def update_special_day(row):
    if pd.isna(row['date']):
        return row['Special_Day']

    release_date = row['date']
    week_start = release_date
    week_end = release_date + timedelta(days=7)

    # Get holidays for release year and year+1 (to catch overlaps)
    year = release_date.year
    all_holidays = []
    for y in [year, year + 1]:
        all_holidays.extend([(name, date) for name, date in get_holidays(y).items() if date is not None])

    # Find matching holidays within the day 1 to +7 day window (includes release date)
    matching_holidays = [
        (name, hol_date) for name, hol_date in all_holidays
        if week_start <= hol_date <= week_end
    ]

    if matching_holidays:
        # Select the one closest to release_date (smallest days difference)
        closest = min(matching_holidays, key=lambda x: (x[1] - release_date).days)
        return closest[0]

    # No match: keep original (e.g., NaN or existing value)
    return row['Special_Day']

# Apply to your DataFrame
# First, ensure 'date' is datetime (if not already)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Update the Special_Day column
df['Special_Day'] = df.apply(update_special_day, axis=1)

# Display to verify
print(df[['tconst', 'date', 'Special_Day']].head())

      tconst       date     Special_Day
0  tt0822832 2009-01-01  New Year's Day
1  tt0960731 2009-01-01  New Year's Day
2  tt0421715 2009-01-01  New Year's Day
3  tt0985699 2009-01-01  New Year's Day
4  tt1068680 2009-01-01  New Year's Day


In [ ]:
df[df['Release']== 'Avatar'] #checking my movie of interest

,tconst,Release,Daily,date,%¬± YD,%¬± LW,Theaters,Release_Year,Avg,Days,To Date,Distributor,genres,runtimeMinutes,Special_Day,directors
16820,tt0499549,Avatar,"$26,752,099",2009-12-18,-,-,"3,452",2009,"$7,749",1,"$26,752,099",Twentieth Century Fox,"Action,Adventure,Fantasy",162,Christmas Day,nm0000116
16868,tt0499549,Avatar,"$25,529,036",2009-12-19,-4.60%,-,"3,452",2009,"$7,395",2,"$52,281,135",Twentieth Century Fox,"Action,Adventure,Fantasy",162,Christmas Day,nm0000116
16916,tt0499549,Avatar,"$24,744,346",2009-12-20,-3.10%,-,"3,452",2009,"$7,168",3,"$77,025,481",Twentieth Century Fox,"Action,Adventure,Fantasy",162,Christmas Day,nm0000116
16963,tt0499549,Avatar,"$16,385,820",2009-12-21,-33.80%,-,"3,452",2009,"$4,746",4,"$93,411,301",Twentieth Century Fox,"Action,Adventure,Fantasy",162,Christmas Day,nm0000116
17007,tt0499549,Avatar,"$16,086,461",2009-12-22,-1.80%,-,"3,452",2009,"$4,660",5,"$109,497,762",Twentieth Century Fox,"Action,Adventure,Fantasy",162,Christmas Day,nm0000116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26734,tt1775309,Avatar,$874,2010-08-04,13.20%,-33.10%,2,2010,$437,230,"$749,750,520",Twentieth Century Fox,Horror,93,NaN,nm3786927
26770,tt1775309,Avatar,"$5,108",2010-08-05,484.40%,36.60%,2,2010,"$2,554",231,"$749,755,628",Twentieth Century Fox,Horror,93,NaN,nm3786927
26815,tt1775309,Avatar,"$2,685",2010-08-06,-47.40%,-33.60%,1,2010,"$2,685",232,"$749,758,313",Twentieth Century Fox,Horror,93,NaN,nm3786927
26858,tt1775309,Avatar,"$3,828",2010-08-07,42.60%,-21.30%,1,2010,"$3,828",233,"$749,762,141",Twentieth Century Fox,Horror,93,NaN,nm3786927


##Calculating Opening Week Revenue from Daily Data

A crucial step for modeling, the sum of 'Daily_Clean' values for the first 7 days of every movie is calculated to get the 'Opening_Week_Revenue'.

In [ ]:
# 1. Clean 'Daily' column: Remove '$' and ',' and convert to numeric
df['Daily_Clean'] = df['Daily'].astype(str).str.replace('$', '').str.replace(',', '')
df['Daily_Clean'] = pd.to_numeric(df['Daily_Clean'], errors='coerce')

# 2. Clean 'Days' column: Remove ',' and convert to numeric
# Errors='coerce' will turn non-numeric values (like '-') into NaN
df['Days_Clean'] = df['Days'].astype(str).str.replace(',', '')
df['Days_Clean'] = pd.to_numeric(df['Days_Clean'], errors='coerce')

# 3. Filter for the first 7 days (Opening Week)
# We select rows where Days is between 1 and 7
opening_week_df = df[(df['Days_Clean'] >= 1) & (df['Days_Clean'] <= 7)]

# 4. Aggregate Revenue and find Release Date
# We group by tconst and Release.
# We sum Daily_Clean to get total opening week revenue.
# We take the minimum of 'date' to get the release date for sorting.
agg_df = opening_week_df.groupby(['tconst', 'Release']).agg(
    Opening_Week_Revenue=('Daily_Clean', 'sum'),
    Release_Date=('date', 'min')
).reset_index()

# 5. Sort by Release_Date chronologically
agg_df = agg_df.sort_values(by='Release_Date', ascending=True)

# Display the first few rows
print(agg_df.head())

        tconst             Release  Opening_Week_Revenue Release_Date
361  tt1034303            Defiance                195622   2009-01-01
231  tt0959337  Revolutionary Road                 76319   2009-01-01
165  tt0841925        Silent Light                 19633   2009-01-07
127  tt0795438   Not Easily Broken               6143981   2009-01-09
207  tt0901476          Bride Wars              25827125   2009-01-09


In [ ]:
agg_df.shape

(4840, 4)

A sanity check was conducted at this point in Excel. Daily earnings of movies in the first 7 days of screening were added for a few movies and matched with the output of above cell. They matched!

##Creating the First Draft

Here begins the first step to creating the feature table for my prediction model. I pick the relevant columns from df and keep them alongside the aggregated revenue column.

In [ ]:
# Selecting the relevant columns from df.
columns_to_merge = ['tconst', 'Theaters', 'Release_Year', 'Distributor', 'genres', 'runtimeMinutes','directors', 'Special_Day']

# Using .drop_duplicates() to ensure unique tconst entries for merging,
# as agg_df has one row per tconst, while df has daily entries for each tconst.
df_columns_for_merge = df[columns_to_merge].drop_duplicates(subset=['tconst'])

# Performing left merge to add these columns to agg_df
agg_df = pd.merge(agg_df, df_columns_for_merge, on='tconst', how='left')

# Checking
print("\n--- Updated agg_df with new columns (Head) ---")
display(agg_df.head())


--- Updated agg_df with new columns (Head) ---


,tconst,Release,Opening_Week_Revenue,Release_Date,Theaters,Release_Year,Distributor,genres,runtimeMinutes,directors,Special_Day
0,tt1034303,Defiance,195622,2009-01-01,2,2009,Paramount Vantage,"Action,Drama,History",137,nm0001880,New Year's Day
1,tt0959337,Revolutionary Road,76319,2009-01-01,3,2009,Paramount Vantage,"Drama,Romance",119,nm0005222,New Year's Day
2,tt0841925,Silent Light,19633,2009-01-07,1,2009,Palisades Tartan,"Drama,Romance",136,nm1196161,NaN
3,tt0795438,Not Easily Broken,6143981,2009-01-09,724,2009,Screen Gems,"Drama,Romance",99,nm0004886,NaN
4,tt0901476,Bride Wars,25827125,2009-01-09,"3,226",2009,Twentieth Century Fox,"Comedy,Romance",89,nm0935095,NaN


Finally, the dataset is extracted into a CSV file for further working in separate codebooks.

In [ ]:
agg_df.to_csv('dimension_table_v1.csv', index=False)
print("\nData saved to 'dimension_table_v1.csv'")


Data saved to 'dimension_table_v1.csv'
